# Выбор месторождения нефти

Заказчик - нефтедобывающая компания. Нужно решить для них, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 100 000 месторождений, где измерили качество нефти и объём её запасов. Требуется построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Требуется проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Условия задачи:**

* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

## Загрузка и подготовка данных

Чтобы изучить данные таблицы, сделать необходимые рассчёты и провести исследование сразу импортируем библиотеки, который нам потребуются в данном проекте:

In [1]:
from io import BytesIO
from sklearn.linear_model import LinearRegression # импортируем алгоритм линейной регрессии библиотеки sklearn  
from sklearn.metrics import mean_squared_error # импортируем функцию для рассчёта среднеквадратичной ошибки
from sklearn.model_selection import train_test_split # импортируем функцию для разбиения датасета на разные типы выборок
from scipy import stats as st # импортируем функцию stats из библиотеки scipy
from scipy.stats import bootstrap  # импортируем функцию bootstrap из библиотеки scipy.stats

import matplotlib.pyplot as plt  # импорт библиотеки matplotlib.pyplot
import numpy as np # импорт библиотеки numpy
import pandas as pd # импорт библиотеки pandas
import requests

Напишем функцию, чтобы прочитать и сохранить данные из Google Sheets

In [2]:
def table(spreadsheet_id):
    spreadsheet_id = spreadsheet_id
    file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
    r = requests.get(file_name)
    table = pd.read_csv(BytesIO(r.content))
    return table

Прочитаем и сохраним файлы, которые предоставил заказчик

In [3]:
df_0 = table('1vz4T7PHK6M5zMtCWtVrYT5inSKoL0WqN') # чтение файла с данными и сохранение в df_0 
df_0.head(10) # получение первых 10 строк таблицы

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


In [4]:
df_1 = table('1Eu11PuatEPl8xh5wHFWrpyN77bHvGQwF') # чтение файла с данными и сохранение в df_1
df_1.head(10) # получение первых 10 строк таблицы 

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


In [5]:
df_2 = table('1P7Mj7VWdS8BVaEd2JC8HnsTgrvleJm-R') # чтение файла с данными и сохранение в df_2
df_2.head(10) # получение первых 10 строк таблицы 

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


Получим общую информацию о таблицах

In [6]:
df_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Посмотрим на данные с использованием метода `describe()`

In [9]:
df_0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [10]:
df_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [11]:
df_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Нам даны 3 таблица по 5 столбцов. Типы данных в столбцах — `float` и `object`.

Согласно документации к данным признаки:
* id — уникальный идентификатор скважины
* f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы)

Целевой признак:
* product — объём запасов в скважине (тыс. баррелей)

**Вывод**
<br><br>
В каждой строке таблицы трёх таблиц — данные о данные геологоразведке скважин. 
<br><br>
Предварительно можно утверждать, что данных достаточно для построения моделей, их настройки, провеки и дальнейшего исследования. 

## Обучение и проверка модели

**Удаление лишних признаков.<br>**
Признак `id` не имеют ценных данных для обучения модели, следовательно его можно удалить:

In [12]:
df_0 = df_0.drop(['id'], axis = 1)
df_0.head(10)

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647
5,0.969570,0.489775,-0.735383,64.741541
6,0.645075,0.530656,1.780266,49.055285
7,-0.400648,0.808337,-5.624670,72.943292
8,0.643105,-0.551583,2.372141,113.356160
9,2.173381,0.563698,9.441852,127.910945


In [13]:
df_1 = df_1.drop(['id'], axis = 1)
df_1.head(10)

,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305
3,-13.081196,-11.506057,4.999415,137.945408
4,12.702195,-8.147433,5.004363,134.766305
5,-3.327590,-2.205276,3.003647,84.038886
6,-11.142655,-10.133399,4.002382,110.992147
7,4.234715,-0.001354,2.004588,53.906522
8,13.355129,-0.332068,4.998647,134.766305
9,1.069227,-11.025667,4.997844,137.945408


In [14]:
df_2 = df_2.drop(['id'], axis = 1)
df_2.head(10)

,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910
3,2.236060,-0.553760,0.930038,114.572842
4,-0.515993,1.716266,5.899011,149.600746
5,-0.758092,0.710691,2.585887,90.222465
6,-0.574891,0.317727,1.773745,45.641478
7,-1.906649,-2.458350,-0.177097,72.480640
8,1.776292,-0.279356,3.004156,106.616832
9,-1.214452,-0.439314,5.922514,52.954532


Для каждого из регионов выполним следующие дествия:
* Разобьём данные на обучающую и валидационную выборки в соотношении 75:25
* Обучим модель Линейной регрессии
* Сохраним предсказания и правильные ответы на валидационной выборке
* Напечатаем на экране средний запас предсказанного сырья и RMSE

**Первый регион:**

In [15]:
train_0, valid_0 = train_test_split(df_0, test_size=0.25, random_state=12345) # разбиваем выборку на обучающую и валидационную
features_train_0 = train_0.drop(['product'], axis=1)
target_train_0 = train_0['product']
features_valid_0 = valid_0.drop(['product'], axis=1)
target_valid_0 = valid_0['product']
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0) # обучаем модель
predictions_0 = model_0.predict(features_valid_0) # делаем предсказание
rmse_0 = mean_squared_error(target_valid_0, predictions_0)**0.5 # находим квадратный корень из средней квадратичной ошибки
predictions_0 = pd.Series(predictions_0) # cохраним предсказания 
target_valid_0 = target_valid_0.reset_index(drop=True) # сохраним правильные ответы на валидационной выборке
print(f'Средний запас предсказанного сырья в данном регионе: {predictions_0.mean() :0.2f}')
print(f'RMSE (квадратный корень из средней квадратичной ошибки): {rmse_0 :0.2f}')

Средний запас предсказанного сырья в данном регионе: 92.59
RMSE (квадратный корень из средней квадратичной ошибки): 37.58


**Второй регион:**

In [16]:
train_1, valid_1 = train_test_split(df_1, test_size=0.25, random_state=12345)# разбиваем выборку на обучающую и валидационную
features_train_1 = train_1.drop(['product'], axis=1)
target_train_1 = train_1['product']
features_valid_1 = valid_1.drop(['product'], axis=1)
target_valid_1 = valid_1['product']
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1) # обучаем модель
predictions_1 = model_1.predict(features_valid_1) # делаем предсказание
rmse_1 = mean_squared_error(target_valid_1, predictions_1)**0.5 # находим квадратный корень из средней квадратичной ошибки
predictions_1 = pd.Series(predictions_1)  # cохраним предсказания 
target_valid_1 = target_valid_1.reset_index(drop=True) # сохраним правильные ответы на валидационной выборке
print(f'Средний запас предсказанного сырья в данном регионе: {predictions_1.mean() :0.2f}')
print(f'RMSE (квадратный корень из средней квадратичной ошибки): {rmse_1 :0.2f}')

Средний запас предсказанного сырья в данном регионе: 68.73
RMSE (квадратный корень из средней квадратичной ошибки): 0.89


**Третий регион:**

In [17]:
train_2, valid_2 = train_test_split(df_2, test_size=0.25, random_state=12345)# разбиваем выборку на обучающую и валидационную
features_train_2 = train_2.drop(['product'], axis=1)
target_train_2 = train_2['product']
features_valid_2 = valid_2.drop(['product'], axis=1)
target_valid_2 = valid_2['product']
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2) # обучаем модель
predictions_2 = model_2.predict(features_valid_2) # делаем предсказание
rmse_2 = mean_squared_error(target_valid_2, predictions_2)**0.5 # находим квадратный корень из средней квадратичной ошибки
predictions_2 = pd.Series(predictions_2)  # cохраним предсказания 
target_valid_2 = target_valid_2.reset_index(drop=True) # сохраним правильные ответы на валидационной выборке
print(f'Средний запас предсказанного сырья в данном регионе: {predictions_2.mean() :0.2f}')
print(f'RMSE (квадратный корень из средней квадратичной ошибки): {rmse_2 :0.2f}')

Средний запас предсказанного сырья в данном регионе: 94.97
RMSE (квадратный корень из средней квадратичной ошибки): 40.03


**Вывод**
<br><br>
Регионы `df_0 и df_2` имеют близкие значения, как среднего запаса предсказанного сырья, так и RNSE, при этом в регионе `df_1` значиельно ниже среднего запаса предсказанного сырья, но RMSE в этом регионе почти нет (меньше 1).
<br> На основании этого можно сделать выводы, что в регионе `df_1` предсказания получились гораздо более точными на валидационной выборке, чем в регионах `df_0 и df_2`.

## Подготовка к расчёту прибыли

Все ключевые значения для расчётов сохраним в отдельных переменных

In [18]:
BUDGET = 10000000000 # бюджет  выделяемый на регион
PRICE_1000_BARRELS = 450000 # доход с каждой единицы продукта, объём указан в тысячах баррелей
WELLS_BEST = 200 # число лучших скважин выбираемых в регионе
WELLS = 500 # число исследуемых скважин

Рассчитаем какой объём в среднем должна иметь каждая скважина из 200 лучших в регионе  

In [19]:
mean_volume_well = BUDGET/WELLS_BEST/PRICE_1000_BARRELS
print(f'Средний объём скважин:{mean_volume_well :0.2f}')

Средний объём скважин:111.11


**Вывод**
<br><br>
Из технического задания и произведённых расчётов можно сделать выводы, что выбрать скважины нужно с объемом запасов выше среднего объёма запасов в любом из регионов. 
<br>По предсказанию моделей средний объём скважин не достигает необходимых нам 111,11 тысяч баррелей в скважине ни в одном из регионов.
<br>Нам нужно будет выбрать скважины, чтобы их объём был выше среднего, чтобы хватило бюджета на разработку скважин в регионе, а в целом нужно получить максимально возможную прибыль.

## Расчёт прибыли

Напишем функцию для расчёта прибыли по региону из выбранных скважин предсказаных моделью

In [20]:
def profit(target, predict, count):
   
    predicted_top_200 = predict.nlargest(200) # возьмём 200 максимальных значений из предсказаний
    target_top_200 = target[predicted_top_200.index][:count] # возьмём целевое значение объёма сырья, соответствующее предсказаниям
    profit = target_top_200.sum()*PRICE_1000_BARRELS - BUDGET # рассчитаем прибыль для полученного объёма сырья
    
    return profit

Посчитаем прибыль для каждого региона

In [21]:
profit_0 = profit(target_valid_0, predictions_0, WELLS_BEST)
profit_1 = profit(target_valid_1, predictions_1, WELLS_BEST)
profit_2 = profit(target_valid_2, predictions_2, WELLS_BEST)
print(f'Прибыль первого региона: {profit_0 :0.2f} \nПрибыль второго региона: {profit_1 :0.2f} \nПрибыль третьего региона:{profit_2 :0.2f}')

Прибыль первого региона: 3320826042.86 
Прибыль второго региона: 2415086696.68 
Прибыль третьего региона:2710349963.60


**Вывод**
<br><br>
В первом регионе по предсказаниям модели получилась самая большая прибыль с учётом цены разработки скважин в 10 млрд.  рублей.
<br>Прибыль первого региона составила 3320826043.14 рублей.

## Расчёт рисков 

Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
<br>Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.

**Первый регион:**

In [22]:
state = np.random.RandomState(12345)
values_0 = []
for i in range(1000):
    target_subsample_0 = target_valid_0.sample(replace=True, random_state=state, n=WELLS)
    probs_subsample_0 = predictions_0[target_subsample_0.index]
    values_0.append(profit(target_subsample_0, probs_subsample_0, WELLS_BEST))
    
values_0 = pd.Series(values_0)
mean_0 = values_0.mean()
lower_0 = values_0.quantile(0.025)
upper_0 = values_0.quantile(0.975)
risk_0 = (values_0 < 0).mean() * 100
print(f'Первый регион: \n--------------\nСредняя прибыль: {mean_0 :0.2f}')
print(f'95-% доверительный интервал: ({lower_0 :0.2f}, {upper_0 :0.2f}) \nРиск убытков: {risk_0}%')

Первый регион: 
--------------
Средняя прибыль: 425938526.91
95-% доверительный интервал: (-102090094.74, 947976353.12) 
Риск убытков: 6.0%


**Второй регион:**

In [23]:
values_1 = []
for i in range(1000):
    target_subsample_1 = target_valid_1.sample(replace=True, random_state=state, n=WELLS)
    probs_subsample_1 = predictions_1[target_subsample_1.index]
    values_1.append(profit(target_subsample_1, probs_subsample_1, WELLS_BEST))
    
values_1 = pd.Series(values_1)
mean_1 = values_1.mean()
lower_1 = values_1.quantile(0.025)
upper_1 = values_1.quantile(0.975)
risk_1 = (values_1 < 0).mean() * 100
print(f'Второй регион: \n--------------\nСредняя прибыль: {mean_1 :0.2f}') 
print(f'95-% доверительный интервал: ({lower_1 :0.2f}, {upper_1 :0.2f}) \nРиск убытков: {risk_1}%')

Второй регион: 
--------------
Средняя прибыль: 518259493.70
95-% доверительный интервал: (128123231.43, 953612982.07) 
Риск убытков: 0.3%


**Третий регион:**

In [24]:
values_2 = []
for i in range(1000):
    target_subsample_2 = target_valid_2.sample(replace=True, random_state=state, n=WELLS)
    probs_subsample_2 = predictions_2[target_subsample_2.index]
    values_2.append(profit(target_subsample_2, probs_subsample_2, WELLS_BEST))
    
values_2 = pd.Series(values_2)
mean_2 = values_2.mean()
lower_2 = values_2.quantile(0.025)
upper_2 = values_2.quantile(0.975)
risk_2 = (values_2 < 0).mean() * 100
print(f'Третий регион: \n--------------\nСредняя прибыль: {mean_2 :0.2f}') 
print(f'95-% доверительный интервал: ({lower_2 :0.2f}, {upper_2 :0.2f}) \nРиск убытков: {risk_2}%')

Третий регион: 
--------------
Средняя прибыль: 420194005.34
95-% доверительный интервал: (-115852609.16, 989629939.84) 
Риск убытков: 6.2%


**Вывод**
<br><br>
**На основании проведённого исследования и расчётов предлагаю производить разработку месторождений во втором регионе.**
<br><br>
Обоснования выбора региона:
<br>Не смотря на то, что в данном регионе прибыль на 200 лучших скважинах наименьшая из трёх регионов и составляет 2415086696.68 рублей, но в данном регионе мы имеем наименьшие риски получить убыток. 
<br>Риск убытка во втором регионе составляет всего 0,3%, что примерно в 20 раз ниже рисков первого и третьего регионов.
<br>
Такие низкие риски связаны с тем, что во втором регионе квадратный корень из средней квадратичной ошибки составляет всего 0.89 тыс. баррелей, что 40-45 раз меньше,  чем в первом и третьем регионе.